# Ultimate Kinematic Gold Standard Pipeline (Step 06)

Unified, production-grade kinematics: root-relative positions, hierarchical dual-track (raw vs zeroed), Savitzky–Golay on quaternions, physiological audit, and hard-proof validation.

**Outputs:** `derivatives/step_06_kinematics/ultimate/{RUN_ID}__kinematics_master.parquet`, `{RUN_ID}__validation_report.json`, and Plotly dashboard.

In [1]:
import os
import sys
import json
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.spatial.transform import Rotation as R
from scipy.signal import savgol_filter
from scipy.stats import pearsonr

if os.path.basename(os.getcwd()) == 'notebooks':
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
else:
    PROJECT_ROOT = os.path.abspath(os.getcwd())
SRC_PATH = os.path.join(PROJECT_ROOT, "src")
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

from pipeline_config import CONFIG
from angular_velocity import compute_omega_and_alpha, compare_angular_velocity_methods
from kinematic_repair import identify_critical_units, apply_surgical_repair

ENFORCE_CLEANING = CONFIG.get("ENFORCE_CLEANING", False)
OMEGA_METHOD = CONFIG.get("OMEGA_METHOD", "quat_log")
OMEGA_FRAME = "local" if (CONFIG.get("OMEGA_FRAME") or "child_body").lower() in ("child_body", "local") else "global"

DERIV_FILTERED = os.path.join(PROJECT_ROOT, CONFIG['derivatives_dir'], "step_04_filtering")
DERIV_REF = os.path.join(PROJECT_ROOT, CONFIG['derivatives_dir'], "step_05_reference")
OUT_DIR = os.path.join(PROJECT_ROOT, CONFIG['derivatives_dir'], "step_06_kinematics", "ultimate")
os.makedirs(OUT_DIR, exist_ok=True)

RUN_ID = Path(CONFIG['current_csv']).stem
FS = float(CONFIG.get('FS_TARGET', 120.0))
SG_WINDOW_SEC = CONFIG.get('SG_WINDOW_SEC', 0.175)
SG_POLYORDER = CONFIG.get('SG_POLYORDER', 3)
dt = 1.0 / FS

INPUT_PARQUET = Path(DERIV_FILTERED) / f"{RUN_ID}__filtered.parquet"
if not INPUT_PARQUET.exists():
    raise FileNotFoundError(f"Step 04 output not found: {INPUT_PARQUET}")

df_in = pd.read_parquet(INPUT_PARQUET)
n_frames = len(df_in)

map_path = Path(DERIV_REF) / f"{RUN_ID}__kinematics_map.json"
if not map_path.exists():
    map_path = Path(PROJECT_ROOT) / CONFIG['derivatives_dir'] / "step_03_resample" / f"{RUN_ID}__kinematics_map.json"
with open(map_path, 'r', encoding='utf-8') as f:
    kinematics_map = json.load(f)

ref_path = Path(DERIV_REF) / f"{RUN_ID}__reference_map.json"
if not ref_path.exists():
    raise FileNotFoundError(f"Reference map not found: {ref_path}")
with open(ref_path, 'r', encoding='utf-8') as f:
    ref_pose = json.load(f)

ROOT_SEGMENT = 'Pelvis' if 'Pelvis' in kinematics_map else 'Hips'

def savgol_window_len(fs, w_sec, polyorder):
    w_len = int(round(w_sec * fs))
    if w_len % 2 == 0:
        w_len += 1
    w_len = max(5, w_len, polyorder + 2)
    if w_len % 2 == 0:
        w_len += 1
    return w_len

W_LEN = savgol_window_len(FS, SG_WINDOW_SEC, SG_POLYORDER)
print(f"RUN_ID: {RUN_ID}, FPS: {FS}, Root: {ROOT_SEGMENT}, Frames: {n_frames}, SavGol window: {W_LEN}")
print(f"ENFORCE_CLEANING: {ENFORCE_CLEANING}")
print(f"OMEGA_METHOD: {OMEGA_METHOD}, OMEGA_FRAME: {OMEGA_FRAME}")

RUN_ID: 763_T2_P2_R2_Take_2025-12-25 10.51.23 AM_005, FPS: 120.0, Root: Hips, Frames: 17262, SavGol window: 21
ENFORCE_CLEANING: False


## Helpers: Quaternion unrolling and renormalization

In [2]:
def unroll_quat(q):
    """Ensure temporal continuity (shortest path); q shape (T, 4)."""
    q = np.asarray(q, dtype=float)
    if q.ndim == 1:
        q = q.reshape(1, -1)
    out = q.copy()
    for i in range(1, len(out)):
        if np.dot(out[i], out[i - 1]) < 0:
            out[i] *= -1
    return out

def renormalize_quat(q):
    """Unit norm per row; q shape (T, 4)."""
    n = np.linalg.norm(q, axis=1, keepdims=True)
    n = np.where(n < 1e-12, 1.0, n)
    return q / n

## Root-relative positions and dual-track quaternion processing

For each joint: hierarchical q_rel = inv(parent)*child → unroll → SavGol smooth → renormalize (Track A). Track B: q_zeroed = inv(q_ref_rel)*q_raw_smooth. Then ω from quat_log, α from SavGol derivative on ω. Linear v,a from root-relative positions.

In [3]:
# Root-relative positions
root_pos_cols = [f"{ROOT_SEGMENT}__px", f"{ROOT_SEGMENT}__py", f"{ROOT_SEGMENT}__pz"]
root_pos = df_in[root_pos_cols].values if all(c in df_in.columns for c in root_pos_cols) else None
pos_cols = [c for c in df_in.columns if c.endswith(('__px', '__py', '__pz')) and df_in[c].notna().all()]
axis_idx = {'__px': 0, '__py': 1, '__pz': 2}
pos_rel = {}
if root_pos is not None:
    for col in pos_cols:
        seg, suffix = col.split('__')[0], '__' + col.split('__')[1]
        idx = axis_idx.get(suffix, 0)
        pos_rel[col] = df_in[col].values - root_pos[:, idx]

# Master result: time_s first
result = {'time_s': df_in['time_s'].values}
validation_rows = []
OMEGA_THRESH = 3000.0
LIN_ACC_THRESH = 20000.0

In [4]:
for joint_name, info in kinematics_map.items():
    parent_name = info.get('parent')
    q_c_cols = [f"{joint_name}__qx", f"{joint_name}__qy", f"{joint_name}__qz", f"{joint_name}__qw"]
    if not all(c in df_in.columns for c in q_c_cols):
        continue
    q_c = df_in[q_c_cols].values
    if parent_name is not None:
        q_p_cols = [f"{parent_name}__qx", f"{parent_name}__qy", f"{parent_name}__qz", f"{parent_name}__qw"]
        if not all(c in df_in.columns for c in q_p_cols):
            continue
        q_p = df_in[q_p_cols].values
        rot_p = R.from_quat(q_p)
        rot_c = R.from_quat(q_c)
        q_rel = (rot_p.inv() * rot_c).as_quat()
    else:
        q_rel = q_c.copy()
    q_rel = unroll_quat(q_rel)
    for ax, i in enumerate(['x', 'y', 'z', 'w']):
        q_rel[:, ax] = savgol_filter(q_rel[:, ax], W_LEN, SG_POLYORDER, mode='interp')
    q_raw_smooth = renormalize_quat(q_rel)
    q_ref_c = np.array([ref_pose[f"{joint_name}__qx"], ref_pose[f"{joint_name}__qy"], ref_pose[f"{joint_name}__qz"], ref_pose[f"{joint_name}__qw"]])
    if parent_name is not None:
        q_ref_p = np.array([ref_pose[f"{parent_name}__qx"], ref_pose[f"{parent_name}__qy"], ref_pose[f"{parent_name}__qz"], ref_pose[f"{parent_name}__qw"]])
        q_rel_ref = (R.from_quat(q_ref_p).inv() * R.from_quat(q_ref_c)).as_quat()
    else:
        q_rel_ref = q_ref_c
    rot_ref = R.from_quat(q_rel_ref)
    q_zeroed = (rot_ref.inv() * R.from_quat(q_raw_smooth)).as_quat()
    q_zeroed = renormalize_quat(q_zeroed)
    omega_raw_rad, alpha_raw_rad = compute_omega_and_alpha(q_raw_smooth, FS, method=OMEGA_METHOD, frame=OMEGA_FRAME, savgol_window=W_LEN, savgol_poly=SG_POLYORDER)
    omega_zeroed_rad, alpha_zeroed_rad = compute_omega_and_alpha(q_zeroed, FS, method=OMEGA_METHOD, frame=OMEGA_FRAME, savgol_window=W_LEN, savgol_poly=SG_POLYORDER)
    omega_raw_deg = np.degrees(omega_raw_rad)
    omega_zeroed_deg = np.degrees(omega_zeroed_rad)
    alpha_raw = np.degrees(alpha_raw_rad)
    alpha_zeroed = np.degrees(alpha_zeroed_rad)
    for ax, letter in enumerate(['x', 'y', 'z']):
        result[f"{joint_name}__raw_rel_omega_{letter}"] = omega_raw_deg[:, ax]
        result[f"{joint_name}__zeroed_rel_omega_{letter}"] = omega_zeroed_deg[:, ax]
    for ax, letter in enumerate(['x', 'y', 'z']):
        result[f"{joint_name}__raw_rel_alpha_{letter}"] = alpha_raw[:, ax]
        result[f"{joint_name}__zeroed_rel_alpha_{letter}"] = alpha_zeroed[:, ax]
    for ax, letter in enumerate(['x', 'y', 'z', 'w']):
        result[f"{joint_name}__raw_rel_q{letter}"] = q_raw_smooth[:, ax]
        result[f"{joint_name}__zeroed_rel_q{letter}"] = q_zeroed[:, ax]
    mag_omega_raw = np.linalg.norm(omega_raw_deg, axis=1)
    mag_omega_zeroed = np.linalg.norm(omega_zeroed_deg, axis=1)
    geodesic_deg = np.degrees(2 * np.arccos(np.clip(np.abs((q_raw_smooth * q_zeroed).sum(axis=1)), 0, 1)))
    vel_align = 100.0 * pearsonr(mag_omega_raw, mag_omega_zeroed)[0] if np.std(mag_omega_raw) > 1e-10 else 100.0
    validation_rows.append({
        'joint': joint_name,
        'geodesic_offset_std': round(float(np.std(geodesic_deg)), 6),
        'velocity_alignment_pct': round(vel_align, 2),
        'max_omega_deg_s': round(float(np.max(mag_omega_raw)), 2),
        'mean_omega_deg_s': round(float(np.mean(mag_omega_raw)), 2),
        'median_omega_deg_s': round(float(np.median(mag_omega_raw)), 2),
        'exceeded_omega_threshold': bool(np.max(mag_omega_raw) > OMEGA_THRESH),
    })
print(f"Processed {len(validation_rows)} joints (dual-track omega, alpha, quaternions).")

Processed 19 joints (dual-track omega, alpha, quaternions).


In [5]:
# Linear velocity and acceleration from root-relative positions (same SavGol)
for col in pos_cols:
    seg, suffix = col.split('__')[0], '__' + col.split('__')[1]
    if col not in pos_rel:
        continue
    pr = pos_rel[col]
    vel = savgol_filter(pr, W_LEN, SG_POLYORDER, deriv=1, delta=dt, mode='interp')
    acc = savgol_filter(pr, W_LEN, SG_POLYORDER, deriv=2, delta=dt, mode='interp')
    axis_letter = col[-1]
    result[f"{seg}__lin_vel_rel_{axis_letter}"] = vel
    result[f"{seg}__lin_acc_rel_{axis_letter}"] = acc
segments_with_pos = set(col.split('__')[0] for col in pos_cols if col in pos_rel)
lin_audit = []
for seg in segments_with_pos:
    cx, cy, cz = f"{seg}__lin_acc_rel_x", f"{seg}__lin_acc_rel_y", f"{seg}__lin_acc_rel_z"
    if cx in result and cy in result and cz in result:
        mag_acc = np.linalg.norm(np.column_stack([result[cx], result[cy], result[cz]]), axis=1)
        lin_audit.append({'segment': seg, 'max_lin_acc_mm_s2': round(float(np.max(mag_acc)), 2), 'exceeded_lin_acc_threshold': bool(np.max(mag_acc) > LIN_ACC_THRESH)})
print(f"Linear v,a computed for {len(pos_cols)} position components; {len(lin_audit)} segments audited.")

Linear v,a computed for 57 position components; 19 segments audited.


## Method Selection Report (mandatory diagnostic)

Methodological justification for using **quat_log** for angular velocity: compare quaternion logarithm vs finite-difference methods. Runs every time to provide an audit trail.

In [6]:
# Method Selection Report: compare quat_log vs 5-point vs central difference (runs every time)
rep_joint = next((j for j in kinematics_map if all(f"{j}__raw_rel_q{c}" in result for c in "xyzw")), None)
if rep_joint is not None:
    q_rep = np.column_stack([result[f"{rep_joint}__raw_rel_qx"], result[f"{rep_joint}__raw_rel_qy"],
                             result[f"{rep_joint}__raw_rel_qz"], result[f"{rep_joint}__raw_rel_qw"]])
    cmp = compare_angular_velocity_methods(q_rep, FS, frame="local")
    stats = cmp["statistics"]
    rec = cmp["method_recommendation"]
    print("Method Selection Report (representative joint:", rep_joint, ")")
    print("  Noise (2nd-deriv std): quat_log = {:.4f}, 5pt = {:.4f}, central = {:.4f}".format(
        stats["noise_qlog"], stats["noise_5pt"], stats["noise_central"]))
    print("  Noise reduction quat_log vs central: {:.2f}x".format(stats["noise_reduction_qlog_vs_central"]))
    print("  Recommendation:", rec)
else:
    print("Method Selection Report: no joint with raw_rel quaternions found; skip.")

Method Selection Report (representative joint: Hips )
  Noise (2nd-deriv std): quat_log = 0.0480, 5pt = 0.0272, central = 0.0326
  Noise reduction quat_log vs central: 0.68x
  Recommendation: 5point_stencil (lowest noise)


## Export master parquet and validation report JSON

In [7]:
df_master = pd.DataFrame(result)
out_parquet = Path(OUT_DIR) / f"{RUN_ID}__kinematics_master.parquet"
df_master.to_parquet(out_parquet, index=False)
print(f"Saved: {out_parquet}")

validation_report = {
    "run_id": RUN_ID,
    "total_frames": len(df_master),
    "per_joint": {r["joint"]: {"geodesic_offset_std": r["geodesic_offset_std"], "velocity_alignment_pct": r["velocity_alignment_pct"], "max_omega_deg_s": r["max_omega_deg_s"], "mean_omega_deg_s": r["mean_omega_deg_s"], "median_omega_deg_s": r["median_omega_deg_s"], "exceeded_omega_threshold": r["exceeded_omega_threshold"]} for r in validation_rows},
    "per_segment_linear": {a["segment"]: {"max_lin_acc_mm_s2": a["max_lin_acc_mm_s2"], "exceeded_lin_acc_threshold": a["exceeded_lin_acc_threshold"]} for a in lin_audit},
}
out_json = Path(OUT_DIR) / f"{RUN_ID}__validation_report.json"
with open(out_json, "w", encoding="utf-8") as f:
    json.dump(validation_report, f, indent=2)
print(f"Saved: {out_json}")
max_geodesic = max(r["geodesic_offset_std"] for r in validation_rows)
mean_align = np.mean([r["velocity_alignment_pct"] for r in validation_rows])
print("Velocity Magnitude Alignment: mean similarity (raw vs zeroed) = {:.2f}% (expect close to 100%).".format(mean_align))
print("Stability Proof (quaternion): max Std of geodesic distance over joints = {:.6f}° (expect near zero; constant T-pose → constant rotation).".format(max_geodesic))
print()
validation_table = pd.DataFrame([{"joint": r["joint"], "velocity_similarity_pct": r["velocity_alignment_pct"], "geodesic_std_deg": r["geodesic_offset_std"]} for r in validation_rows])
print("Validation Summary Table (quaternion-based proof for proceeding with zeroed data):")
display(validation_table)
if mean_align >= 99.0 and np.isfinite(max_geodesic) and max_geodesic < 0.1:
    print("Conclusion: T-pose is stable; zeroed data aligns with raw. Proceed using zeroed data.")
else:
    print("Conclusion: Review T-pose or per-joint metrics above before relying solely on zeroed data.")

Saved: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_06_kinematics\ultimate\763_T2_P2_R2_Take_2025-12-25 10.51.23 AM_005__kinematics_master.parquet
Saved: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_06_kinematics\ultimate\763_T2_P2_R2_Take_2025-12-25 10.51.23 AM_005__validation_report.json
Velocity Magnitude Alignment: mean similarity (raw vs zeroed) = 100.00% (expect close to 100%).
Stability Proof (quaternion): max Std of geodesic distance over joints = 0.000000° (expect near zero; constant T-pose → constant rotation).

Validation Summary Table (quaternion-based proof for proceeding with zeroed data):


,joint,velocity_similarity_pct,geodesic_std_deg
0,Hips,100.0,0.0
1,Spine,100.0,0.0
2,Spine1,100.0,0.0
3,Neck,100.0,0.0
4,Head,100.0,0.0
5,LeftUpLeg,100.0,0.0
6,LeftLeg,100.0,0.0
7,LeftFoot,100.0,0.0
8,RightUpLeg,100.0,0.0
9,RightLeg,100.0,0.0


Conclusion: T-pose is stable; zeroed data aligns with raw. Proceed using zeroed data.


## Outlier validation: frame counts and % by metric (3-tier: Warning / Alert / Critical)

Per-metric tables: joints as rows; columns = outlier frame counts and percentage of total recording frames for each threshold level.

In [8]:
total_frames = len(df_master)
THRESH = {
    'rotation_mag_deg': {'WARNING': 140.0, 'ALERT': 160.0, 'CRITICAL': 180.0},
    'angular_velocity_deg_s': {'WARNING': 800.0, 'ALERT': 1200.0, 'CRITICAL': 1500.0},
    'angular_acceleration_deg_s2': {'WARNING': 35000.0, 'ALERT': 50000.0, 'CRITICAL': 80000.0},
    'linear_velocity_mm_s': {'WARNING': 3000.0, 'ALERT': 5000.0, 'CRITICAL': 7000.0},
    'linear_acceleration_mm_s2': {'WARNING': 60000.0, 'ALERT': 100000.0, 'CRITICAL': 150000.0},
}

def outlier_table(joint_or_segment_list, magnitude_per_name, thresh_dict, total_frames, name_col='joint'):
    rows = []
    for name in joint_or_segment_list:
        mag = magnitude_per_name.get(name)
        if mag is None or len(mag) == 0:
            continue
        n_w = int(np.sum(mag > thresh_dict['WARNING']))
        n_a = int(np.sum(mag > thresh_dict['ALERT']))
        n_c = int(np.sum(mag > thresh_dict['CRITICAL']))
        rows.append({
            name_col: name,
            'n_frames_WARNING': n_w, 'n_frames_ALERT': n_a, 'n_frames_CRITICAL': n_c,
            'pct_WARNING': round(100.0 * n_w / total_frames, 2),
            'pct_ALERT': round(100.0 * n_a / total_frames, 2),
            'pct_CRITICAL': round(100.0 * n_c / total_frames, 2),
            'total_frames': total_frames,
        })
    return pd.DataFrame(rows)

# 1. Rotation vector magnitude (total rotation from T-pose) — zeroed quat
rot_mag_per_joint = {}
for j in kinematics_map:
    qc = [f"{j}__zeroed_rel_qx", f"{j}__zeroed_rel_qy", f"{j}__zeroed_rel_qz", f"{j}__zeroed_rel_qw"]
    if not all(c in df_master.columns for c in qc):
        continue
    q = df_master[qc].values
    rot_mag_per_joint[j] = np.degrees(np.linalg.norm(R.from_quat(q).as_rotvec(), axis=1))
df_rot = outlier_table(list(rot_mag_per_joint.keys()), rot_mag_per_joint, THRESH['rotation_mag_deg'], total_frames)
print("Rotation magnitude (deg) — thresholds 140° / 160° / 180°")
display(df_rot)

# 2. Angular velocity (deg/s)
omega_mag_per_joint = {}
for j in kinematics_map:
    cols = [f"{j}__zeroed_rel_omega_x", f"{j}__zeroed_rel_omega_y", f"{j}__zeroed_rel_omega_z"]
    if not all(c in df_master.columns for c in cols):
        continue
    omega_mag_per_joint[j] = np.linalg.norm(df_master[cols].values, axis=1)
df_omega = outlier_table(list(omega_mag_per_joint.keys()), omega_mag_per_joint, THRESH['angular_velocity_deg_s'], total_frames)
print("Angular velocity (deg/s) — thresholds 800 / 1200 / 1500")
display(df_omega)

# 3. Angular acceleration (deg/s²)
alpha_mag_per_joint = {}
for j in kinematics_map:
    cols = [f"{j}__zeroed_rel_alpha_x", f"{j}__zeroed_rel_alpha_y", f"{j}__zeroed_rel_alpha_z"]
    if not all(c in df_master.columns for c in cols):
        continue
    alpha_mag_per_joint[j] = np.linalg.norm(df_master[cols].values, axis=1)
df_alpha = outlier_table(list(alpha_mag_per_joint.keys()), alpha_mag_per_joint, THRESH['angular_acceleration_deg_s2'], total_frames)
print("Angular acceleration (deg/s²) — thresholds 35000 / 50000 / 80000")
display(df_alpha)

# 4. Linear velocity (mm/s) — per segment
lin_vel_segments = [c.split('__')[0] for c in df_master.columns if c.endswith('__lin_vel_rel_x')]
lin_vel_mag = {}
for seg in lin_vel_segments:
    cx, cy, cz = f"{seg}__lin_vel_rel_x", f"{seg}__lin_vel_rel_y", f"{seg}__lin_vel_rel_z"
    if cx in df_master.columns and cy in df_master.columns and cz in df_master.columns:
        lin_vel_mag[seg] = np.linalg.norm(df_master[[cx, cy, cz]].values, axis=1)
df_linvel = outlier_table(list(lin_vel_mag.keys()), lin_vel_mag, THRESH['linear_velocity_mm_s'], total_frames, name_col='segment')
print("Linear velocity (mm/s) — thresholds 3000 / 5000 / 7000")
display(df_linvel)

# 5. Linear acceleration (mm/s²)
lin_acc_segments = [c.split('__')[0] for c in df_master.columns if c.endswith('__lin_acc_rel_x')]
lin_acc_mag = {}
for seg in lin_acc_segments:
    cx, cy, cz = f"{seg}__lin_acc_rel_x", f"{seg}__lin_acc_rel_y", f"{seg}__lin_acc_rel_z"
    if cx in df_master.columns and cy in df_master.columns and cz in df_master.columns:
        lin_acc_mag[seg] = np.linalg.norm(df_master[[cx, cy, cz]].values, axis=1)
df_linacc = outlier_table(list(lin_acc_mag.keys()), lin_acc_mag, THRESH['linear_acceleration_mm_s2'], total_frames, name_col='segment')
print("Linear acceleration (mm/s²) — thresholds 60000 / 100000 / 150000")
display(df_linacc)

# --- Surgical repair (when ENFORCE_CLEANING and any Critical): delegate to kinematic_repair ---
from kinematic_repair import identify_critical_units, apply_surgical_repair

angular_critical_joints, linear_critical_segments = identify_critical_units(
    df_rot, df_omega, df_alpha, df_linvel, df_linacc
)

if ENFORCE_CLEANING and (angular_critical_joints or linear_critical_segments):
    apply_surgical_repair(
        result,
        pos_rel,
        validation_rows,
        lin_audit,
        angular_critical_joints,
        linear_critical_segments,
        rot_mag_per_joint,
        omega_mag_per_joint,
        alpha_mag_per_joint,
        lin_vel_mag,
        lin_acc_mag,
        THRESH,
        kinematics_map,
        ref_pose,
        FS,
        W_LEN,
        SG_POLYORDER,
        dt,
        OMEGA_THRESH,
        LIN_ACC_THRESH,
    )
    # (Angular/linear repair done in-place by kinematic_repair; now save and rebuild outlier tables)
    df_master = pd.DataFrame(result)
    out_parquet = Path(OUT_DIR) / f"{RUN_ID}__kinematics_master.parquet"
    df_master.to_parquet(out_parquet, index=False)
    print("Surgical repair applied; overwrote", out_parquet)
    validation_report = {"run_id": RUN_ID, "total_frames": len(df_master), "per_joint": {r["joint"]: {"geodesic_offset_std": r["geodesic_offset_std"], "velocity_alignment_pct": r["velocity_alignment_pct"], "max_omega_deg_s": r["max_omega_deg_s"], "mean_omega_deg_s": r["mean_omega_deg_s"], "median_omega_deg_s": r["median_omega_deg_s"], "exceeded_omega_threshold": r["exceeded_omega_threshold"]} for r in validation_rows}, "per_segment_linear": {a["segment"]: {"max_lin_acc_mm_s2": a["max_lin_acc_mm_s2"], "exceeded_lin_acc_threshold": a["exceeded_lin_acc_threshold"]} for a in lin_audit}}
    out_json = Path(OUT_DIR) / f"{RUN_ID}__validation_report.json"
    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(validation_report, f, indent=2)
    print("Overwrote", out_json)
    # Rebuild outlier tables from repaired df_master
    rot_mag_per_joint = {j: np.degrees(np.linalg.norm(R.from_quat(df_master[[f"{j}__zeroed_rel_qx", f"{j}__zeroed_rel_qy", f"{j}__zeroed_rel_qz", f"{j}__zeroed_rel_qw"]].values).as_rotvec(), axis=1)) for j in kinematics_map if all(f"{j}__zeroed_rel_q{c}" in df_master.columns for c in "xyzw")}
    omega_mag_per_joint = {j: np.linalg.norm(df_master[[f"{j}__zeroed_rel_omega_x", f"{j}__zeroed_rel_omega_y", f"{j}__zeroed_rel_omega_z"]].values, axis=1) for j in kinematics_map if all(f"{j}__zeroed_rel_omega_{c}" in df_master.columns for c in "xyz")}
    alpha_mag_per_joint = {j: np.linalg.norm(df_master[[f"{j}__zeroed_rel_alpha_x", f"{j}__zeroed_rel_alpha_y", f"{j}__zeroed_rel_alpha_z"]].values, axis=1) for j in kinematics_map if all(f"{j}__zeroed_rel_alpha_{c}" in df_master.columns for c in "xyz")}
    lin_vel_mag = {seg: np.linalg.norm(df_master[[f"{seg}__lin_vel_rel_x", f"{seg}__lin_vel_rel_y", f"{seg}__lin_vel_rel_z"]].values, axis=1) for seg in lin_vel_segments if all(f"{seg}__lin_vel_rel_{c}" in df_master.columns for c in "xyz")}
    lin_acc_mag = {seg: np.linalg.norm(df_master[[f"{seg}__lin_acc_rel_x", f"{seg}__lin_acc_rel_y", f"{seg}__lin_acc_rel_z"]].values, axis=1) for seg in lin_acc_segments if all(f"{seg}__lin_acc_rel_{c}" in df_master.columns for c in "xyz")}
    df_rot = outlier_table(list(rot_mag_per_joint.keys()), rot_mag_per_joint, THRESH["rotation_mag_deg"], total_frames)
    df_omega = outlier_table(list(omega_mag_per_joint.keys()), omega_mag_per_joint, THRESH["angular_velocity_deg_s"], total_frames)
    df_alpha = outlier_table(list(alpha_mag_per_joint.keys()), alpha_mag_per_joint, THRESH["angular_acceleration_deg_s2"], total_frames)
    df_linvel = outlier_table(list(lin_vel_mag.keys()), lin_vel_mag, THRESH["linear_velocity_mm_s"], total_frames, name_col="segment")
    df_linacc = outlier_table(list(lin_acc_mag.keys()), lin_acc_mag, THRESH["linear_acceleration_mm_s2"], total_frames, name_col="segment")

# Save outlier validation tables to JSON (standalone and into validation_report)
outlier_report = {
    "run_id": RUN_ID,
    "total_frames": total_frames,
    "thresholds": THRESH,
    "rotation_mag_deg": df_rot.to_dict(orient="records"),
    "angular_velocity_deg_s": df_omega.to_dict(orient="records"),
    "angular_acceleration_deg_s2": df_alpha.to_dict(orient="records"),
    "linear_velocity_mm_s": df_linvel.to_dict(orient="records"),
    "linear_acceleration_mm_s2": df_linacc.to_dict(orient="records"),
}
outlier_json_path = Path(OUT_DIR) / f"{RUN_ID}__outlier_validation.json"
with open(outlier_json_path, "w", encoding="utf-8") as f:
    json.dump(outlier_report, f, indent=2)
print(f"Saved: {outlier_json_path}")

# Append outlier_validation (frame counts and percentages) to validation_report.json
validation_report_path = Path(OUT_DIR) / f"{RUN_ID}__validation_report.json"
if validation_report_path.exists():
    with open(validation_report_path, "r", encoding="utf-8") as f:
        v = json.load(f)
    v["total_frames"] = total_frames
    v["outlier_validation"] = {
        "total_frames": total_frames,
        "thresholds": THRESH,
        "rotation_mag_deg": df_rot.to_dict(orient="records"),
        "angular_velocity_deg_s": df_omega.to_dict(orient="records"),
        "angular_acceleration_deg_s2": df_alpha.to_dict(orient="records"),
        "linear_velocity_mm_s": df_linvel.to_dict(orient="records"),
        "linear_acceleration_mm_s2": df_linacc.to_dict(orient="records"),
    }
    with open(validation_report_path, "w", encoding="utf-8") as f:
        json.dump(v, f, indent=2)
    print(f"Updated: {validation_report_path} (total_frames + outlier_validation with n_frames_* and pct_*)")

Rotation magnitude (deg) — thresholds 140° / 160° / 180°


,joint,n_frames_WARNING,n_frames_ALERT,n_frames_CRITICAL,pct_WARNING,pct_ALERT,pct_CRITICAL,total_frames
0,Hips,1222,618,0,7.08,3.58,0.0,17262
1,Spine,0,0,0,0.00,0.00,0.0,17262
2,Spine1,0,0,0,0.00,0.00,0.0,17262
3,Neck,0,0,0,0.00,0.00,0.0,17262
4,Head,0,0,0,0.00,0.00,0.0,17262
5,LeftUpLeg,0,0,0,0.00,0.00,0.0,17262
6,LeftLeg,0,0,0,0.00,0.00,0.0,17262
7,LeftFoot,0,0,0,0.00,0.00,0.0,17262
8,RightUpLeg,24,0,0,0.14,0.00,0.0,17262
9,RightLeg,11,0,0,0.06,0.00,0.0,17262


Angular velocity (deg/s) — thresholds 800 / 1200 / 1500


,joint,n_frames_WARNING,n_frames_ALERT,n_frames_CRITICAL,pct_WARNING,pct_ALERT,pct_CRITICAL,total_frames
0,Hips,0,0,0,0.00,0.00,0.0,17262
1,Spine,0,0,0,0.00,0.00,0.0,17262
2,Spine1,0,0,0,0.00,0.00,0.0,17262
3,Neck,0,0,0,0.00,0.00,0.0,17262
4,Head,0,0,0,0.00,0.00,0.0,17262
5,LeftUpLeg,0,0,0,0.00,0.00,0.0,17262
6,LeftLeg,29,0,0,0.17,0.00,0.0,17262
7,LeftFoot,7,0,0,0.04,0.00,0.0,17262
8,RightUpLeg,0,0,0,0.00,0.00,0.0,17262
9,RightLeg,13,0,0,0.08,0.00,0.0,17262


Angular acceleration (deg/s²) — thresholds 35000 / 50000 / 80000


...

Linear velocity (mm/s) — thresholds 3000 / 5000 / 7000


,segment,n_frames_WARNING,n_frames_ALERT,n_frames_CRITICAL,pct_WARNING,pct_ALERT,pct_CRITICAL,total_frames
0,RightHand,753,0,0,4.36,0.00,0.0,17262
1,RightLeg,8,0,0,0.05,0.00,0.0,17262
2,Neck,0,0,0,0.00,0.00,0.0,17262
3,LeftHand,204,8,0,1.18,0.05,0.0,17262
4,RightForeArm,115,0,0,0.67,0.00,0.0,17262
5,RightUpLeg,0,0,0,0.00,0.00,0.0,17262
6,RightArm,0,0,0,0.00,0.00,0.0,17262
7,RightFoot,34,0,0,0.20,0.00,0.0,17262
8,Hips,0,0,0,0.00,0.00,0.0,17262
9,Spine,0,0,0,0.00,0.00,0.0,17262


Linear acceleration (mm/s²) — thresholds 60000 / 100000 / 150000


,segment,n_frames_WARNING,n_frames_ALERT,n_frames_CRITICAL,pct_WARNING,pct_ALERT,pct_CRITICAL,total_frames
0,RightHand,0,0,0,0.0,0.0,0.0,17262
1,RightLeg,0,0,0,0.0,0.0,0.0,17262
2,Neck,0,0,0,0.0,0.0,0.0,17262
3,LeftHand,0,0,0,0.0,0.0,0.0,17262
4,RightForeArm,0,0,0,0.0,0.0,0.0,17262
5,RightUpLeg,0,0,0,0.0,0.0,0.0,17262
6,RightArm,0,0,0,0.0,0.0,0.0,17262
7,RightFoot,0,0,0,0.0,0.0,0.0,17262
8,Hips,0,0,0,0.0,0.0,0.0,17262
9,Spine,0,0,0,0.0,0.0,0.0,17262


Saved: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_06_kinematics\ultimate\763_T2_P2_R2_Take_2025-12-25 10.51.23 AM_005__outlier_validation.json
Updated: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_06_kinematics\ultimate\763_T2_P2_R2_Take_2025-12-25 10.51.23 AM_005__validation_report.json (total_frames + outlier_validation with n_frames_* and pct_*)


## Plotly dashboard: 3D skeleton (slider + animation), Raw vs Zeroed ω, Geodesic stability

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Build root-relative position arrays per segment for skeleton
seg_xyz = {}
for seg in segments_with_pos:
    kx, ky, kz = f"{seg}__px", f"{seg}__py", f"{seg}__pz"
    if kx in pos_rel and ky in pos_rel and kz in pos_rel:
        seg_xyz[seg] = np.column_stack([pos_rel[kx], pos_rel[ky], pos_rel[kz]])
n_f = len(df_in)
t_axis = df_in['time_s'].values

# Edges from kinematics_map (parent -> child) where both have position
edges = []
for jname, jinfo in kinematics_map.items():
    pname = jinfo.get("parent")
    if pname is None:
        continue
    if jname in seg_xyz and pname in seg_xyz:
        edges.append((pname, jname))

def skeleton_trace(frame_ix):
    scatter = go.Scatter3d(
        x=[seg_xyz[s][frame_ix, 0] for s in seg_xyz],
        y=[seg_xyz[s][frame_ix, 1] for s in seg_xyz],
        z=[seg_xyz[s][frame_ix, 2] for s in seg_xyz],
        mode='markers+text', text=list(seg_xyz.keys()), textposition='top center', marker=dict(size=4, color='blue'),
        name='segments'
    )
    line_x, line_y, line_z = [], [], []
    for p, c in edges:
        line_x += [seg_xyz[p][frame_ix, 0], seg_xyz[c][frame_ix, 0], None]
        line_y += [seg_xyz[p][frame_ix, 1], seg_xyz[c][frame_ix, 1], None]
        line_z += [seg_xyz[p][frame_ix, 2], seg_xyz[c][frame_ix, 2], None]
    bone = go.Scatter3d(x=line_x, y=line_y, z=line_z, mode='lines', line=dict(color='gray', width=2), name='bones')
    return [scatter, bone]

n_slider = min(150, n_f)
step = max(1, (n_f - 1) // n_slider)
frame_indices = list(range(0, n_f, step))
if frame_indices[-1] != n_f - 1:
    frame_indices.append(n_f - 1)
fig_skel = go.Figure(data=skeleton_trace(0))
fig_skel.update_layout(
    title=f'Root-relative skeleton — {RUN_ID}',
    scene=dict(
        xaxis_title='X (mm) — floor',
        yaxis_title='Y (mm) — up',
        zaxis_title='Z (mm) — floor',
        aspectmode='data',
        camera=dict(up=dict(x=0, y=1, z=0), eye=dict(x=1.2, y=1.2, z=1.2)),
    ),
    updatemenus=[dict(type='buttons', showactive=False, y=0.1, buttons=[dict(label='Play', method='animate', args=[None, dict(frame=dict(duration=40, redraw=True), fromcurrent=True)])])],
    sliders=[dict(active=0, len=0.9, x=0.1, steps=[dict(args=[[str(i)], dict(frame=dict(duration=0), mode='immediate')], label=str(frame_indices[i]), method='animate') for i in range(len(frame_indices))])]
)
fig_skel.frames = [go.Frame(data=skeleton_trace(fi), name=str(i)) for i, fi in enumerate(frame_indices)]
if seg_xyz and edges:
    fig_skel.show()
else:
    print("No root-relative positions or edges; skipping 3D skeleton.")

In [10]:
# Comparison: Raw vs Zeroed omega magnitude; Geodesic distance over time (one joint)
rep_joint = next((j for j in kinematics_map if f"{j}__raw_rel_omega_x" in result), list(kinematics_map.keys())[0])
mag_raw = np.linalg.norm(np.column_stack([result[f"{rep_joint}__raw_rel_omega_x"], result[f"{rep_joint}__raw_rel_omega_y"], result[f"{rep_joint}__raw_rel_omega_z"]]), axis=1)
mag_zeroed = np.linalg.norm(np.column_stack([result[f"{rep_joint}__zeroed_rel_omega_x"], result[f"{rep_joint}__zeroed_rel_omega_y"], result[f"{rep_joint}__zeroed_rel_omega_z"]]), axis=1)
q_raw = np.column_stack([result[f"{rep_joint}__raw_rel_qx"], result[f"{rep_joint}__raw_rel_qy"], result[f"{rep_joint}__raw_rel_qz"], result[f"{rep_joint}__raw_rel_qw"]])
q_zeroed = np.column_stack([result[f"{rep_joint}__zeroed_rel_qx"], result[f"{rep_joint}__zeroed_rel_qy"], result[f"{rep_joint}__zeroed_rel_qz"], result[f"{rep_joint}__zeroed_rel_qw"]])
geodesic_deg = np.degrees(2 * np.arccos(np.clip(np.abs((q_raw * q_zeroed).sum(axis=1)), 0, 1)))

fig_comp = make_subplots(rows=2, cols=1, subplot_titles=(f'{rep_joint}: Raw vs Zeroed ω magnitude (°/s)', 'Geodesic distance (°) — should be flat'))
fig_comp.add_trace(go.Scatter(x=t_axis, y=mag_raw, name='raw_rel ω', line=dict(color='blue')), row=1, col=1)
fig_comp.add_trace(go.Scatter(x=t_axis, y=mag_zeroed, name='zeroed_rel ω', line=dict(color='orange')), row=1, col=1)
fig_comp.add_trace(go.Scatter(x=t_axis, y=geodesic_deg, name='geodesic', line=dict(color='green')), row=2, col=1)
fig_comp.update_layout(height=500, title_text=f'Validation: {rep_joint}')
fig_comp.update_xaxes(title_text='Time (s)', row=2, col=1)
fig_comp.show()